In [1]:
install.packages("survival", dependencies=TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
install.packages("abind")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("rmatio")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("rmatio"):
“installation of package ‘rmatio’ had non-zero exit status”


In [4]:
install.packages("BART")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("BART"):
“installation of package ‘BART’ had non-zero exit status”


In [ ]:
unzip("/content/data01.zip", exdir = "/content")

In [5]:
library(BART)

Loading required package: nlme

Loading required package: survival



In [6]:
library(rmatio)

In [7]:
library(abind)

In [ ]:
i_exps <- c(0:9)
dataset = "Flickr"
dataset_extra = "0.5"
x_str = ""

trs <- c(0.8)
ts = 0.2
# val = 0.0001

set.seed(42)

for (tr in trs){
    df = data.frame()
    for (i_exp in i_exps) {
        fn <- paste("/content/data", dataset, sep="/")
        fn <- paste(fn, dataset_extra, sep="")
        fn <- paste(fn, "", sep="/")
        fn <- paste(fn, dataset, sep="")
        fn <- paste(fn, toString(i_exp), sep="")
        fn <- paste(fn, ".mat", sep="")
        filename <- system.file(fn, package='rmatio')
        print(i_exp)
        mat <- read.mat(fn)
    #     print(m)
        if(x_str=="full"){
            X <- as.matrix(mat$Attributes)
        }
        else{
            X <- as.matrix(mat$X_100)
        }

        n = dim(X)[1]
        T <- mat$T
        Y1 <- mat$Y1
        Y0 <- mat$Y0
        A <- mat$Network
    #     print(typeof(X))
        XT <- abind(X,T,along=2)
        XT_CF <- abind(X,1-T,along=2)
        YF <- ifelse(T > 0, Y1, Y0)
        YCF <- ifelse(T > 0, Y0, Y1)

        #split the data
        print(n)
        idx = 1:n
        sample(idx)

        n_tr = floor(tr*n)
        n_ts = floor(ts*n)
        # n_val = floor(val*n)

    #     print(n)
    #     print(n_tr)
    #     print(n_ts)
    #     print(n_val)
    #     print(idx)

#         idx_tr = idx[1:n_tr]
#         idx_val = idx[(n-n_val-n_ts):(n-n_ts)]
#         idx_ts = idx[(n-n_ts):(n)]

        idx_tr = idx[1:n_tr]
        idx_ts = idx[(n_tr):(n_ts+n_tr)]

    #     print(idx_ts)
        my_bart_f = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT[idx_ts,], nskip=5, ndpost=5)
        my_bart_cf = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT_CF[idx_ts,], nskip=5, ndpost=5)

        my_bart_f_in = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT[idx_tr,])
        my_bart_cf_in = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT_CF[idx_tr,])

#         yf_ts = YF[idx_ts]
        yf_pred_ts = my_bart_f$yhat.test.mean
        ycf_pred_ts = my_bart_cf$yhat.test.mean

        y1_pred_ts = ifelse(T[idx_ts]>0, yf_pred_ts, ycf_pred_ts)
        y0_pred_ts = ifelse(T[idx_ts]>0, ycf_pred_ts, yf_pred_ts)

    #     print(dim(y1_pred_ts))

        pehe_out = sqrt(mean(((y1_pred_ts-y0_pred_ts)-(Y1-Y0)[idx_ts])**2))
        mae_ate_out = abs(mean((y1_pred_ts-y0_pred_ts))-mean((Y1-Y0)[idx_ts]))
        # df <- rbind(df, data.frame(pehe,mae_ate))

        # yf_pred_ts_in = my_bart_cfin$yhat.train.mean
        yf_pred_ts_in = my_bart_f_in$yhat.test.mean
        ycf_pred_ts_in = my_bart_cf_in$yhat.test.mean

        y1_pred_ts_in = ifelse(T[idx_tr]>0, yf_pred_ts_in, ycf_pred_ts_in)
        y0_pred_ts_in = ifelse(T[idx_tr]>0, ycf_pred_ts_in, yf_pred_ts_in)

        pehe_in = sqrt(mean(((y1_pred_ts_in-y0_pred_ts_in)-(Y1-Y0)[idx_tr])**2))
        mae_ate_in = abs(mean((y1_pred_ts_in-y0_pred_ts_in))-mean((Y1-Y0)[idx_tr]))

        df <- rbind(df, data.frame(mae_ate_in, pehe_in, mae_ate_out, pehe_out))


    }
    ofn <- paste('/content/data/',dataset,sep="")
    ofn <- paste(ofn, dataset_extra, sep="")
    ofn <- paste(ofn, toString(tr), sep="/")
    ofn <- paste(ofn, x_str, sep="")
    ofn <- paste(ofn, ".csv", sep="")
    write.csv(df, ofn, row.names=F)
}


In [10]:
i_exps <- c(0:9)
dataset = "Flickr"
dataset_extra = "0.5"
x_str = ""

trs <- c(0.8)
ts = 0.2
# val = 0.0001

set.seed(42)


for (tr in trs){
    df = data.frame()
    for (i_exp in i_exps) {
        fn <- paste("/content/data", dataset, sep="/")
        fn <- paste(fn, dataset_extra, sep="")
        fn <- paste(fn, "", sep="/")
        fn <- paste(fn, dataset, sep="")
        fn <- paste(fn, toString(i_exp), sep="")
        fn <- paste(fn, ".mat", sep="")
        filename <- system.file(fn, package='rmatio')
        print(i_exp)
        mat <- read.mat(fn)
    #     print(m)
        if(x_str=="full"){
            X <- as.matrix(mat$Attributes)
        }
        else{
            X <- as.matrix(mat$X_100)
        }

        n = dim(X)[1]
        T <- mat$T
        Y1 <- mat$Y1
        Y0 <- mat$Y0
        A <- mat$Network
    #     print(typeof(X))
        XT <- abind(X,T,along=2)
        XT_CF <- abind(X,1-T,along=2)
        YF <- ifelse(T > 0, Y1, Y0)
        YCF <- ifelse(T > 0, Y0, Y1)

        #split the data
        print(n)
        idx = 1:n
        sample(idx)

        n_tr = floor(tr*n)
        n_ts = floor(ts*n)
        # n_val = floor(val*n)

    #     print(n)
    #     print(n_tr)
    #     print(n_ts)
    #     print(n_val)
    #     print(idx)

#         idx_tr = idx[1:n_tr]
#         idx_val = idx[(n-n_val-n_ts):(n-n_ts)]
#         idx_ts = idx[(n-n_ts):(n)]

        idx_tr = idx[1:n_tr]
        idx_ts = idx[(n_tr):(n_ts+n_tr)]

    #     print(idx_ts)
        my_bart_f = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT, nskip=5, ndpost=5)
        my_bart_cf = wbart(XT[idx_tr,], YF[idx_tr], x.test=XT_CF, nskip=5, ndpost=5)

#         yf_ts = YF[idx_ts]
        yf_pred_ts = my_bart_f$yhat.test.mean
        ycf_pred_ts = my_bart_cf$yhat.test.mean

        y1_pred_ts = ifelse(T>0, yf_pred_ts, ycf_pred_ts)
        y0_pred_ts = ifelse(T>0, ycf_pred_ts, yf_pred_ts)

    #     print(dim(y1_pred_ts))

        pehe_out = sqrt(mean(((y1_pred_ts-y0_pred_ts)[idx_ts]-(Y1-Y0)[idx_ts])**2))
        mae_ate_out = abs(mean((y1_pred_ts-y0_pred_ts)[idx_ts])-mean((Y1-Y0)[idx_ts]))

        pehe_in = sqrt(mean(((y1_pred_ts-y0_pred_ts)[idx_tr]-(Y1-Y0)[idx_tr])**2))
        mae_ate_in = abs(mean((y1_pred_ts-y0_pred_ts)[idx_tr])-mean((Y1-Y0)[idx_tr]))

        df <- rbind(df, data.frame(mae_ate_in, pehe_in, mae_ate_out, pehe_out))


    }
    ofn <- paste('/content/data/',dataset,sep="")
    ofn <- paste(ofn, dataset_extra, sep="")
    ofn <- paste(ofn, toString(tr), sep="/")
    ofn <- paste(ofn, x_str, sep="")
    ofn <- paste(ofn, ".csv", sep="")
    write.csv(df, ofn, row.names=F)
}


[1] 0
[1] 7575
*****Into main of wbart
*****Data:
data:n,p,np: 6060, 1195, 7575
y1,yn: 85.749008, -6.574596
x1,x[n*p]: 0.000000, 1.000000
xp1,xp[np*p]: 0.000000, 1.000000
*****Number of Trees: 200
*****Number of Cut Points: 5 ... 1
*****burn and ndpost: 5, 5
*****Prior:beta,alpha,tau,nu,lambda: 2.000000,0.950000,4.148860,3.000000,33.457813
*****sigma: 13.105809
*****w (weights): 1.000000 ... 1.000000
*****Dirichlet:sparse,theta,omega,a,b,rho,augment: 0,0,1,0.5,1,1195,0
*****nkeeptrain,nkeeptest,nkeeptestme,nkeeptreedraws: 5,5,5,5
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1

MCMC
done 0 (out of 10)
time: 1s
check counts
trcnt,tecnt,temecnt,treedrawscnt: 5,5,5,5
*****Into main of wbart
*****Data:
data:n,p,np: 6060, 1195, 7575
y1,yn: 85.749008, -6.574596
x1,x[n*p]: 0.000000, 1.000000
xp1,xp[np*p]: 0.000000, 0.000000
*****Number of Trees: 200
*****Number of Cut Points: 5 ... 1
*****burn and ndpost: 5, 5
*****Prior:beta,alpha,tau,nu,lambda: 2.000000,0.950000,4.14